<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/classifiers/rule_ml_pair_pron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pymorphy2[fast]

     |████████████████████████████████| 61kB 4.4MB/s 
     |████████████████████████████████| 8.2MB 11.1MB/s 
     |████████████████████████████████| 378kB 32.3MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=860520 sha256=aa398b67aa383b7c770a6a84d79203ee1fb2f7f896109fe8c9654c7e8445ab9e
  Stored in directory: /root/.cache/pip/wheels/3d/1f/f0/a5b1f9d02e193c997d252c33d215f24dfd7a448bc0166b2a12
Successfully built DAWG


## **Импорт**

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [3]:
from IPython.display import SVG, display, HTML

In [4]:
from google.colab import drive
drive.mount('./content', force_remount=False)

Mounted at ./content


In [5]:
import spacy
import numpy as np

import gensim
from gensim.models import Word2Vec 
from gensim.models import FastText

from nltk.tokenize import RegexpTokenizer
from nltk import  sent_tokenize
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
from graphviz import Source
from sklearn import tree, svm, linear_model
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


In [7]:
!ls
import os
for i in os.listdir('content/'):
  print(i)

content  sample_data
MyDrive
.shortcut-targets-by-id
.file-revisions-by-id
.Trash


In [8]:
import csv
import re
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

## **Обработка данных**

In [9]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/hv-info_gepatit2.txt

--2021-05-28 22:13:51--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11655409 (11M) [text/plain]
Saving to: ‘neuroleptic_antidepr2.txt’

neuroleptic_antidep 100%[===================>]  11.12M  37.3MB/s    in 0.3s    

2021-05-28 22:13:52 (37.3 MB/s) - ‘neuroleptic_antidepr2.txt’ saved [11655409/11655409]

--2021-05-28 22:13:52--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443.

In [10]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/annotation_table.csv

--2021-05-28 22:13:55--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-05-28 22:13:55 ERROR 404: Not Found.

--2021-05-28 22:13:55--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/annotation_table.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2100469 (2.0M) [text/plain]
Saving to: ‘annotation_table.csv’

annotation_table.cs 100%[===================>]   2.00M  --.-KB/s    in 0.07

In [11]:
handler = open('neuroleptic_antidepr2.txt', 'r', encoding='utf-8')
my_txt = handler.read()
handler.close()

handler = open('hv-info_gepatit2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

handler = open('rakpobedim_ru2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

In [12]:
posts = my_txt.split('----')

In [13]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [14]:
morph.parse('он')[0].normal_form

'он'

## **Обучение векторизатора**

In [15]:
tokenizer = RegexpTokenizer(r'\w+')
sentences_tokenized = [w.lower() for w in posts]


sentences_tokenized = [tokenizer.tokenize(morph.parse(i)[0].normal_form) for i in sentences_tokenized]
ft = FastText(sentences_tokenized, window=5, min_count=5, workers=4,sg=1)

Обработка битых символов, чтобы pandas мог открыть таблицу.

In [16]:
count = 0
with open('annotation_table.csv') as csvfile:
  with open('annotations.csv', 'w') as write_file:
    writer = csv.writer(write_file, delimiter='\t')
    for new_row in csvfile:
      count +=1
      # print(i.split('\t'))
      if len(new_row.split('\t')) > 19:
        new_row = new_row.split('\t')[:20]
        new_row[-1] = new_row[-1][:-1] # cut \n
      writer.writerow(new_row)
count

27019

In [17]:
# # with open('coref_df_anns60k.csv') as csvfile:
# with open('new_anns60k.csv') as csvfile:
#   reader = csv.reader(csvfile, delimiter='\t')
#   with open('new_df_anns60k.csv', 'w') as write_file:
#     writer = csv.writer(write_file, delimiter='\t')
#     c = 0
#     for row in reader:
#       c +=1
#       new_row = [i if i and not re.match(r'\t+?', i) else 0 for i in row]
#       if len(new_row) > 9:
#         new_row = new_row[:9]
#       else:
#         new_row = [0 for i in range(9)]
#       writer.writerow(new_row)
# c


In [18]:
# !rm new_anns60k.csv.1 new_anns60k.csv.2 new_df_anns60k.csv new_anns60k.csv
!ls
# !rm annotation_table.csv.2 annotation_table.csv.1 annotation_table.csv annotations.csv

annotations.csv       hv-info_gepatit2.txt	 sample_data
annotation_table.csv  neuroleptic_antidepr2.txt
content		      rakpobedim_ru2.txt


In [19]:
# anns_df = pd.read_csv('coref_df_anns60k.csv', delimiter='\t', error_bad_lines=False, header=None)
anns_df = pd.read_csv('annotations.csv', delimiter='\t')
anns_df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,plur
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,2,0.0,sing
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27013,и,0,22,68.0,0.0,2385.0,NaN,NaN,NaN,NaN,CONJ,cc,пошлины,inan,nomn,femn,NOUN,0,0.0,NaN
27014,таможенные,0,22,68.0,0.0,2386.0,NaN,nomn,NaN,NaN,ADJF,amod,пошлины,inan,nomn,femn,NOUN,0,0.0,plur
27015,пошлины,0,22,68.0,0.0,2387.0,inan,nomn,femn,NaN,NOUN,nsubj,есть,NaN,NaN,NaN,INFN,0,0.0,plur
27016,как,0,22,68.0,0.0,2388.0,NaN,NaN,NaN,NaN,CONJ,advmod,есть,NaN,NaN,NaN,INFN,0,0.0,NaN


In [20]:
tok_list = []
for token in anns_df.TOKEN:
  try:
    tok_list.append(ft.wv.__getitem__(token.lower()))
  except (KeyError, AttributeError):
    tok_list.append(0)
    # print(token, 'here')
anns_df['TOKEN_VECT'] = tok_list


tok_list = []
for token in anns_df.HEAD:
  try:
    tok_list.append(ft.wv.__getitem__(token.lower()))
  except (KeyError, AttributeError):
    tok_list.append(0)
    # print(token, 'here')
anns_df['HEAD_VECT'] = tok_list

In [21]:
anns_df = anns_df.loc[(anns_df['TOPIC_NUM'].isin(list(range(30)))) | (anns_df['SENT_NUM'].isin(list(range(10000))))]
anns_df = anns_df.reset_index().drop(['index'], axis=1)

In [22]:
topics = list(set(anns_df.TOPIC_NUM.values))
topics
sent_vect_list = []
# номера предложений топика
for topic_num in topics:

  sents_in_topic = list(set(anns_df.loc[anns_df['TOPIC_NUM'] == topic_num].SENT_NUM))
  topic_len = len(anns_df.loc[anns_df['TOPIC_NUM'] == topic_num])
  for sent_num in sents_in_topic:
    tok_vects = anns_df.loc[(anns_df['TOPIC_NUM'] == topic_num) & (anns_df['SENT_NUM'] == sent_num)].TOKEN_VECT
    try:
      sent_vect = sum(list(tok_vects))/len(tok_vects)
    except ZeroDivisionError:
      sent_vect = 0
      print(anns_df.loc[(anns_df['SENT_NUM'] == sent_num)])
    for i in range(len(tok_vects)):
      sent_vect_list.append(sent_vect)
len(sent_vect_list)
# если не класть нулевых предложений - будет длина датафрейма, как он раньше импортировлся, но поздно.

26497

In [23]:
anns_df['SENT_VECT'] = sent_vect_list

Проверим данные

In [24]:
anns_df.SENT_NUM[18]

1.0

# **VECTORS HANDLING**

Изменяем значения векторов предложений, главного слова группы, зависимых слов группы, и токена на косинусную дистанцию с семантической осью.

Сначала, изменим строковое представление на массив.

Добавим размеченные данные, и посмотрим, что получилось.

Ограничим датафрейм теми данными, которые были размечены, чтобы была возможность провести оценку системы.

In [25]:
# df = anns_df.iloc[:15000,:]
# df = anns_df.iloc[:17000,:]
df = anns_df.iloc[:18000,:]

Всего наименований препаратов размечено (это не обязательно антецеденты, любое название препарата).

In [26]:
len(df.loc[df['NER'] == 1])

628

Из них местоимений (прономинальные анафоры).

In [27]:
len(df.loc[df['NER'] == 3])

117

И номинативные анафоры существительные, такие как "препарат" и "лекарство". В основном это они и есть.

In [28]:
len(df.loc[df['NER'] == 2])

246

In [174]:
len(df.loc[(df['NER'].isin([2,3]))])

363

In [30]:
df.loc[df['COREFERENCE_CLUSTER']==0]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,plur,"[0.31147233, 0.57117313, 0.04815721, -0.409445...","[0.31147233, 0.57117313, 0.04815721, -0.409445...","[0.13332218, 0.45600295, 0.29921648, 0.0133830..."
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,2,0.0,sing,"[-0.4082495, 1.2474589, 0.16071026, 0.3023012,...","[0.31147233, 0.57117313, 0.04815721, -0.409445...","[0.13332218, 0.45600295, 0.29921648, 0.0133830..."
5,не,0,1,1.0,0.0,5.0,NaN,NaN,NaN,NaN,PRCL,advmod,знаю,NaN,NaN,NaN,VERB,0,0.0,NaN,"[-0.2695854, 0.13300261, -0.57894105, -0.03271...","[0.43641022, -0.38678503, 0.36828604, -0.13947...","[0.13332218, 0.45600295, 0.29921648, 0.0133830..."
6,знаю,0,1,1.0,0.0,6.0,NaN,NaN,NaN,1per,VERB,ROOT,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,"[0.43641022, -0.38678503, 0.36828604, -0.13947...","[0.43641022, -0.38678503, 0.36828604, -0.13947...","[0.13332218, 0.45600295, 0.29921648, 0.0133830..."
7,стоило,0,1,1.0,0.0,7.0,NaN,NaN,neut,NaN,VERB,ccomp,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,"[0.3855081, 0.03888781, 0.5437503, 0.14336586,...","[0.43641022, -0.38678503, 0.36828604, -0.13947...","[0.13332218, 0.45600295, 0.29921648, 0.0133830..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,схемы,0,21,6.0,0.0,214.0,inan,gent,femn,NaN,NOUN,obj,использовать,NaN,NaN,NaN,INFN,0,0.0,sing,"[1.1061966, 0.18641233, -0.6417462, 0.11630453...","[-0.5316417, 0.008584103, -0.15146814, -0.6833...","[0.035589144, 0.36206114, 0.27995455, -0.03619..."
17996,рекомендованные,0,21,6.0,0.0,215.0,NaN,nomn,NaN,NaN,PRTF,acl,схемы,inan,gent,femn,NOUN,0,0.0,plur,"[0.33841532, 0.11918936, 0.07324212, -0.413560...","[1.1061966, 0.18641233, -0.6417462, 0.11630453...","[0.035589144, 0.36206114, 0.27995455, -0.03619..."
17997,в,0,21,6.0,0.0,216.0,NaN,NaN,NaN,NaN,PREP,case,версиях,inan,loct,femn,NOUN,0,0.0,NaN,"[-0.066019215, 0.04333604, 0.0640202, 0.218741...","[-0.14383897, -0.039206997, 0.33006278, 0.4937...","[0.035589144, 0.36206114, 0.27995455, -0.03619..."
17998,более,0,21,6.0,0.0,217.0,NaN,NaN,NaN,NaN,ADVB,advmod,ранних,NaN,gent,NaN,ADJF,0,0.0,NaN,"[-0.0140536465, -0.20464624, 0.58457786, -0.39...","[0.34023288, -0.13272311, 0.76155555, 0.217555...","[0.035589144, 0.36206114, 0.27995455, -0.03619..."


In [31]:
# df['NER'] = [int(i) for i in df['NER']]
# df['COREFERENCE_CLUSTER'] = [int(i) for i in df['COREFERENCE_CLUSTER']]

Посмотрим на данные.

In [32]:
df.loc[df['NER'].isin([1,2,3])]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,2,0.0,sing,"[-0.4082495, 1.2474589, 0.16071026, 0.3023012,...","[0.31147233, 0.57117313, 0.04815721, -0.409445...","[0.13332218, 0.45600295, 0.29921648, 0.0133830..."
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.5526839, 0.82293695, 0.44306058, -0.2024548...","[-0.4082495, 1.2474589, 0.16071026, 0.3023012,...","[0.13332218, 0.45600295, 0.29921648, 0.0133830..."
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur,"[0.30506766, 0.8536718, 0.38608164, -0.3469684...","[0.5526839, 0.82293695, 0.44306058, -0.2024548...","[0.13332218, 0.45600295, 0.29921648, 0.0133830..."
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing,"[0.49791038, 0.6804126, 0.42184398, -0.1447514...","[0.30506766, 0.8536718, 0.38608164, -0.3469684...","[0.13332218, 0.45600295, 0.29921648, 0.0133830..."
21,лекарством,0,1,1.0,1.0,21.0,inan,ablt,neut,NaN,NOUN,conj,нет,NaN,NaN,NaN,PRED,2,0.0,sing,"[-0.28583363, 0.8644155, 0.4034036, 0.16512683...","[-0.128564, 0.333242, 0.9391225, -0.05283584, ...","[0.13332218, 0.45600295, 0.29921648, 0.0133830..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17918,вел,0,21,4.0,2.0,137.0,NaN,NaN,masc,NaN,VERB,ROOT,вел,NaN,NaN,masc,VERB,1,136.0,sing,"[0.99129164, 0.32085434, 0.77375877, 0.1337012...","[0.99129164, 0.32085434, 0.77375877, 0.1337012...","[0.06997077, 0.38361007, 0.288438, -0.03590363..."
17929,соф,0,21,4.0,3.0,148.0,inan,gent,femn,NaN,NOUN,nmod,форума,inan,gent,masc,NOUN,1,148.0,plur,"[1.1338569, -0.20935594, 0.28536195, -0.928845...","[1.1041255, 0.89730436, 0.037059546, -0.496586...","[0.06997077, 0.38361007, 0.288438, -0.03590363..."
17930,даке,0,21,4.0,3.0,149.0,inan,datv,femn,NaN,NOUN,nmod,соф,inan,gent,femn,NOUN,1,149.0,sing,"[0.3731596, 0.008505226, -0.17235856, -0.50071...","[1.1338569, -0.20935594, 0.28536195, -0.928845...","[0.06997077, 0.38361007, 0.288438, -0.03590363..."
17937,соф,0,21,4.0,3.0,156.0,inan,gent,femn,NaN,NOUN,nsubj,вел,NaN,NaN,masc,VERB,1,156.0,plur,"[1.1338569, -0.20935594, 0.28536195, -0.928845...","[0.99129164, 0.32085434, 0.77375877, 0.1337012...","[0.06997077, 0.38361007, 0.288438, -0.03590363..."


# **SEMANTIC AXIS**

Реализация метода семантических осей.
Сначала, выбираем первые предложения каждого топика. Посмотрим на них.

In [33]:
axis_df = df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0')
axis_df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.5526839, 0.82293695, 0.44306058, -0.2024548...","[-0.4082495, 1.2474589, 0.16071026, 0.3023012,...","[0.13332218, 0.45600295, 0.29921648, 0.0133830..."
576,зелдокс,0,2,0.0,0.0,2.0,inan,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.42491958, 0.45135576, -0.07145757, 0.027565...","[-0.4082495, 1.2474589, 0.16071026, 0.3023012,...","[0.081449576, 0.33994588, 0.31454352, -0.07690..."
1870,оланзапин,0,3,0.0,0.0,2.0,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,plur,"[0.6313993, 0.81830496, 0.13835016, -0.2686472...","[-0.4082495, 1.2474589, 0.16071026, 0.3023012,...","[0.14779967, 0.339999, 0.28284252, -0.03064249..."
5555,галоперидол,0,4,0.0,0.0,2.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,1,2.0,sing,"[0.32613516, 1.0911746, 0.010839594, -0.081562...","[0.31147233, 0.57117313, 0.04815721, -0.409445...","[0.17428152, 0.42232317, 0.13264184, 0.0235525..."
6200,тиаприд,0,5,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,3.0,sing,"[0.5628008, 0.5154813, 0.41394433, -0.17624523...","[-0.4082495, 1.2474589, 0.16071026, 0.3023012,...","[-0.024516322, 0.35105503, 0.096207924, 0.1020..."
8450,сероквель,0,6,0.0,0.0,2.0,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,plur,"[0.45022792, 1.1768608, 0.053721827, 0.1744385...","[-0.4082495, 1.2474589, 0.16071026, 0.3023012,...","[0.02717207, 0.35304126, 0.16138056, -0.116211..."
8541,день,0,7,0.0,0.0,2.0,inan,accs,masc,NaN,NOUN,ROOT,день,inan,accs,masc,NOUN,0,0.0,sing,"[-0.69571257, 0.83386785, -0.8985834, -0.03574...","[-0.69571257, 0.83386785, -0.8985834, -0.03574...","[-0.04881798, 0.4108755, 0.35371187, -0.373367..."
11117,500,0,8,0.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,nummod,мг,inan,nomn,masc,NOUN,0,0.0,NaN,"[-0.34692985, 0.56729627, 0.40729672, 0.021465...","[0.021246888, 0.48612282, 0.5749718, -0.016608...","[0.1367877, 0.24314038, 0.18793495, -0.0593619..."
11787,капецитабин,0,10,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,ROOT,капецитабин,anim,nomn,masc,NOUN,1,2.0,sing,"[0.9200813, 0.89533186, 0.18841232, -0.1400131...","[0.9200813, 0.89533186, 0.18841232, -0.1400131...","[0.10521937, 0.22208403, 0.20404212, -0.014950..."
11806,иресса,0,11,0.0,0.0,2.0,NaN,NaN,NaN,NaN,GRND,nmod,отдам,NaN,NaN,NaN,VERB,1,2.0,NaN,"[0.21933177, 0.6861596, 0.702355, -0.17867197,...","[0.5741573, -0.4165255, 0.107837655, -0.084820...","[0.10521937, 0.22208403, 0.20404212, -0.014950..."


In [34]:
# sum(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)/len(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)

Суммируем верктора токенов, главных слов, предложения целиком, и зависимых слов. Нормализуем.

In [35]:

token_axe = sum(axis_df.TOKEN_VECT)/len(axis_df.TOKEN_VECT)
head_axe = sum(axis_df.HEAD_VECT)/len(axis_df.HEAD_VECT)
# head_childs_axe = sum(axis_df.HEAD_CHILDS_VECT)/len(axis_df.HEAD_CHILDS_VECT)
sent_axe = sum(axis_df.SENT_VECT)/len(axis_df.SENT_VECT)

In [36]:
df.TOKEN_VECT[710]

0

Функция, которая заменяет столбец с векторами на столбец со значениями косинусной дистанции с соответствующей семантической осью.


In [37]:
def make_similarity(df, col, axe):
  new_col = []
  for ind, vect in enumerate(df[col]):
    try:
      new_col.append(cosine_similarity([axe], [vect])[-1][-1])
    except ValueError:
      new_col.append(0)
  df[col] = new_col



In [38]:
make_similarity(df, 'TOKEN_VECT', token_axe)
make_similarity(df, 'HEAD_VECT', head_axe)
# make_similarity(df, 'HEAD_CHILDS_VECT', head_childs_axe)
make_similarity(df, 'SENT_VECT', sent_axe)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [39]:
df.TOKEN[1]

'препарат'

In [40]:
# df['SUM_VECT']= [(df.TOKEN_VECT[i] + df.SENT_VECT[i])/3 for i in range(len(df.SENT_VECT))]

Посмотрим на обработанные данные.

In [41]:
df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,plur,0.507672,0.544900,0.944452
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,2,0.0,sing,0.578114,0.544900,0.944452
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,0.741848,0.856890,0.944452
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur,0.769299,0.561147,0.944452
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing,0.742337,0.620767,0.944452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,схемы,0,21,6.0,0.0,214.0,inan,gent,femn,NaN,NOUN,obj,использовать,NaN,NaN,NaN,INFN,0,0.0,sing,0.465665,0.552159,0.989458
17996,рекомендованные,0,21,6.0,0.0,215.0,NaN,nomn,NaN,NaN,PRTF,acl,схемы,inan,gent,femn,NOUN,0,0.0,plur,0.594935,0.409324,0.989458
17997,в,0,21,6.0,0.0,216.0,NaN,NaN,NaN,NaN,PREP,case,версиях,inan,loct,femn,NOUN,0,0.0,NaN,0.454485,0.451674,0.989458
17998,более,0,21,6.0,0.0,217.0,NaN,NaN,NaN,NaN,ADVB,advmod,ранних,NaN,gent,NaN,ADJF,0,0.0,NaN,0.525261,0.501876,0.989458


# **ПРАВИЛОВЫЙ АЛГОРИТМ**

In [42]:
# df.SENT_NUM = [int(i) for i in df['SENT_NUM']]
# df.POST_NUM = [int(i) for i in df['POST_NUM']]
# # df.TOKEN_NUM = [float(i) for i in df['TOKEN_NUM']]
df.TOPIC_NUM = [int(i) for i in df['TOPIC_NUM']]
df.IS_ANSWER = [int(i) for i in df['TOPIC_NUM']]
# df.NER = [int(i) for i in df['NER']]

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [43]:
topics = list(set(df.TOPIC_NUM.values))
topics

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

In [44]:
df.loc[df['TOKEN_NUM'] == 344 ]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
344,рисполепта,1,1,2.0,4.0,344.0,inan,nomn,femn,NaN,NOUN,nmod,приеме,inan,loct,masc,NOUN,1,344.0,sing,0.745356,0.502140,0.981266
918,вышло,2,2,6.0,3.0,344.0,NaN,NaN,neut,NaN,VERB,ROOT,вышло,NaN,NaN,neut,VERB,0,0.0,sing,0.545427,0.536421,0.985002
2212,в,3,3,5.0,8.0,344.0,NaN,NaN,NaN,NaN,PREP,case,использовании,inan,loct,neut,NOUN,0,0.0,NaN,0.454485,0.541432,0.979064
5897,и,4,4,6.0,7.0,344.0,NaN,NaN,NaN,NaN,CONJ,cc,выписывались,NaN,NaN,NaN,VERB,0,0.0,NaN,0.456090,0.510650,0.917238
6542,в,5,5,17.0,0.0,344.0,NaN,NaN,NaN,NaN,PREP,case,мозгу,inan,loc2,masc,NOUN,0,0.0,NaN,0.454485,0.429606,0.891702
8883,их,7,7,0.0,32.0,344.0,NaN,accs,NaN,3per,NPRO,obj,принимать,NaN,NaN,NaN,INFN,0,0.0,plur,0.459208,0.484840,0.878083
11513,отвратно,9,9,0.0,39.0,344.0,NaN,NaN,NaN,NaN,ADVB,advmod,перенесла,NaN,NaN,femn,VERB,0,0.0,NaN,0.534001,0.511078,0.989252
12936,будет,15,15,0.0,31.0,344.0,NaN,NaN,NaN,3per,VERB,advcl,проходить,NaN,NaN,NaN,INFN,0,0.0,sing,0.474939,0.614640,0.990015
15278,ковид,16,16,9.0,0.0,344.0,inan,nomn,masc,NaN,NOUN,conj,наименование,inan,nomn,neut,NOUN,0,0.0,sing,0.615988,0.523555,0.950848
16465,сказал,18,18,7.0,2.0,344.0,NaN,NaN,masc,NaN,VERB,acl:relcl,парень,anim,nomn,masc,NOUN,0,0.0,sing,0.451085,0.500157,0.986028


In [45]:

topic_num = 1
topic_df = df.loc[df['TOPIC_NUM'] == topic_num]
# номера предложений топика
sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num ].SENT_NUM))
sents_in_topic
# достать пост
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1)]
# достать конкретное предложение конкретного поста
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)]
# достать токен из предложение конкретного поста
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 0)]


,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
5,не,1,1,1.0,0.0,5.0,NaN,NaN,NaN,NaN,PRCL,advmod,знаю,NaN,NaN,NaN,VERB,0,0.0,NaN,0.464174,0.498227,0.944452
6,знаю,1,1,1.0,0.0,6.0,NaN,NaN,NaN,1per,VERB,ROOT,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,0.538441,0.498227,0.944452
7,стоило,1,1,1.0,0.0,7.0,NaN,NaN,neut,NaN,VERB,ccomp,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,0.483714,0.498227,0.944452
8,ли,1,1,1.0,0.0,8.0,NaN,NaN,NaN,NaN,CONJ,advmod,стоило,NaN,NaN,neut,VERB,0,0.0,NaN,0.517284,0.439430,0.944452
9,мне,1,1,1.0,0.0,9.0,NaN,datv,NaN,1per,NPRO,iobj,стоило,NaN,NaN,neut,VERB,0,0.0,sing,0.542550,0.439430,0.944452
10,писать,1,1,1.0,0.0,10.0,NaN,NaN,NaN,NaN,INFN,csubj,стоило,NaN,NaN,neut,VERB,0,0.0,NaN,0.505434,0.439430,0.944452


In [46]:
for i in df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 0)].values:
  print(i[-5])

0.0
0.0
0.0
0.0
0.0
0.0


In [47]:
# анафоры
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)
& (df['NER'] > 1)].values[0]

array(['лекарством', 1, 1, 1.0, 1.0, 21.0, 'inan', 'ablt', 'neut', nan,
       'NOUN', 'conj', 'нет', nan, nan, nan, 'PRED', 2, 0.0, 'sing',
       0.475286066532135, 0.5959170460700989, 0.9444522857666016],
      dtype=object)

In [48]:
def rule_eval(my_X, my_y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(my_X)):
    if my_X[i] != 0:
      if my_X[i] == my_y[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif my_X[i] == 0:
      if my_X[i] == my_y[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')


In [49]:
df.loc[(df['POS'] == 5) &  (df['PERSON'] == 1)].iloc[30:40, :]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT


In [50]:
topics = list(set(df.TOPIC_NUM.values))
topics
# all_anaphors = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['POS'] == 5)]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

In [51]:
for_factorize = 'ANIMACY 	CASE	GENDER	PERSON	POS	DEPENDENCY	HEAD	HEAD_ANIMACY	HEAD_CASE	HEAD_GENDER	HEAD_POS NUMBER'.split()
for_factorize

mapping_dict = {}
for col_name in for_factorize:
  col, lbl = pd.factorize(df[col_name])
  df[col_name] = col
  mapping_dict[col_name] = lbl
# mapping_dict  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [52]:
# def anaphore_in_sent(sent_num, topic_num) это потом
# изначально отталкиваемся от номеров топиков, итерация по ним
# topics = list(set(df.TOPIC_NUM.values)) это выше
anf_len = 0
ant_len = 0
ann_clusters = []
my_clusters = []
post_num = 0
for topic_num in topics:
  # print(topics, 'topics')
  # if post_num == 3:
  #   break
  # извлекаем посты топика
  posts_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].POST_NUM))
  # sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].SENT_NUM))


  for post_num in posts_in_topic:
    sents_in_post = list(set(df.loc[(df['TOPIC_NUM'] == topic_num) \
                                    & (df['POST_NUM'] == post_num)].SENT_NUM))
    # if post_num == 3:
    #   break

    for sent_num in sents_in_post:

      sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num) \
                    & (df['POST_NUM'] == post_num)]
      # собираем анафоры и антецеденты
      # current_anaphors = sent.loc[(sent['NER'] > 1)]
      current_anaphors = sent.loc[(sent['POS'] == 5) &  (sent['NER'] > 1)]
      current_antecedents = sent.loc[(sent['NER'] == 1)]
      anf_len += len(current_anaphors)
      ant_len += len(current_antecedents)
      # если нет анафоров, продолжать не имеет смысла
      
      if not list(current_anaphors.TOKEN):
        continue

      # если в предложении есть один антецедент, самый простой случай
      if len(list(current_antecedents.TOKEN)) >= 1:
        for anfr in current_anaphors.values:
          # проверяем сходимость по числу
          if anfr[-4] == current_antecedents.NUMBER.values[0]:
            # добавляем в список размеченный класс
            ann_clusters.append(anfr[-5])
            # и класс который определило правило(антецедента из предложения)
            my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
          else:
            # в противном случае присваиваем нулевой класс
            ann_clusters.append(anfr[-5])
            # и класс, который определило правило(антецедента из предложения)
            my_clusters.append(0.0)

      # если больше одного!!
          
      # если в предложении его нет =(
      elif len(list(current_antecedents.TOKEN)) == 0:
        # ищем в посте в предложениях выше
        post = df.loc[(df['TOPIC_NUM'] == topic_num) \
                      & (df['POST_NUM'] == post_num) \
                      & (df['SENT_NUM'] <= sent_num)]

        current_antecedents = post.loc[(post['NER'] == 1) ]

        # если его нету в посте, выбираем первый же антецедент из предыдущих постов
        # топика(сверху вниз)
        if not list(current_antecedents.TOKEN):
          topic = df.loc[(df['TOPIC_NUM'] == topic_num)]
          current_antecedents = topic.loc[(topic['NER'] == 1) ]
          # проверяем, а антецеденты-то есть?
          if not current_antecedents.TOKEN.values.any():
              # присваиваем нулевой класс, если нет даже в топике
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              # для оценки
              my_clusters.append(0.0)
              continue
          # ну и если они есть
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу
            # try:
            if anfr[-4] == current_antecedents.NUMBER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
              # print(ann_clusters, my_clusters, 'am I here?')
            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)


        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу, если число вообще есть
          
            if anfr[-4] == current_antecedents.NUMBER.values[-1]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)






      # print(list(current_anaphors.TOKEN), list(current_anaphors.TOKEN), list(current_anaphors.COREFERENCE_CLUSTER) )
      # print(list(current_antecedents.TOKEN), list(current_antecedents.TOKEN), list(current_antecedents.COREFERENCE_CLUSTER) )
print(my_clusters)
print('---')
print(ann_clusters)
print('---')
print(len(my_clusters), len(ann_clusters))
print(anf_len, ant_len)

[2.0, 540.0, 11.0, 11.0, 108.0, 182.0, 182.0, 182.0, 182.0, 658.0, 658.0, 829.0, 829.0, 829.0, 829.0, 0.0, 0.0, 0.0, 0.0, 2.0, 679.0, 0.0, 0.0, 0.0, 1483.0, 0.0, 1750.0, 2031.0, 2053.0, 0.0, 0.0, 0.0, 3125.0, 3270.0, 3619.0, 5.0, 114.0, 174.0, 365.0, 0.0, 527.0, 10.0, 27.0, 41.0, 41.0, 2.0, 135.0, 135.0, 218.0, 239.0, 256.0, 270.0, 270.0, 2.0, 2.0, 447.0, 521.0, 521.0, 521.0, 0.0, 0.0, 0.0, 2.0, 0.0, 1455.0, 1516.0, 1609.0, 1657.0, 1722.0, 1770.0, 0.0, 1905.0, 1984.0, 0.0, 0.0, 2071.0, 0.0, 164.0, 0.0, 410.0, 534.0, 0.0, 0.0, 707.0, 707.0, 801.0, 0.0, 871.0, 881.0, 1350.0, 0.0, 0.0, 0.0, 122.0, 165.0, 263.0, 513.0, 556.0, 654.0, 813.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1867.0, 2138.0, 2295.0, 305.0, 0.0, 0.0, 0.0, 0.0, 752.0]
---
[2.0, 536.0, 11.0, 11.0, 108.0, 182.0, 182.0, 182.0, 182.0, 658.0, 658.0, 829.0, 829.0, 829.0, 829.0, 183.0, 183.0, 183.0, 388.0, 446.0, 686.0, 778.0, 987.0, 987.0, 1490.0, 1582.0, 1750.0, 2031.0, 2053.0, 2344.0, 3.0, 3.0, 3125.0, 3270.0, 

In [53]:
rule_eval(my_clusters, ann_clusters)

52  true positive
26  false positive
14  true negative
28  false negative
---------
0.6666666666666666  presicion
0.65  recall
0.6582278481012659  f-score


# **МОДЕЛИ.**

Факторизуем троковые данные, чтобы вместо строковых представлений классов были численные.

In [54]:
for_factorize = 'ANIMACY 	CASE	GENDER	PERSON	POS	DEPENDENCY	HEAD	HEAD_ANIMACY	HEAD_CASE	HEAD_GENDER	HEAD_POS NUMBER'.split()
for_factorize


['ANIMACY',
 'CASE',
 'GENDER',
 'PERSON',
 'POS',
 'DEPENDENCY',
 'HEAD',
 'HEAD_ANIMACY',
 'HEAD_CASE',
 'HEAD_GENDER',
 'HEAD_POS',
 'NUMBER']

In [55]:
_, lbl = pd.factorize(df.ANIMACY)
print(lbl, )
_

Int64Index([-1, 0, 1], dtype='int64')


array([0, 1, 2, ..., 0, 0, 0])

Делаем словарь с маппингом, чтобы можно было достать нужное значение по индексу. В значении -1 хранится None

In [56]:
mapping_dict = {}
for col_name in for_factorize:
  col, lbl = pd.factorize(df[col_name])
  df[col_name] = col
  mapping_dict[col_name] = lbl
mapping_dict  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


{'ANIMACY': Int64Index([-1, 0, 1], dtype='int64'),
 'CASE': Int64Index([-1, 0, 1, 2, 3, 4, 5, 6, 7, 8], dtype='int64'),
 'DEPENDENCY': Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
             17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
             34, 35],
            dtype='int64'),
 'GENDER': Int64Index([-1, 0, 1, 2], dtype='int64'),
 'HEAD': Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
             ...
             2972, 2973, 2974, 2975, 2976, 2977, 2978, 2979, 2980, 2981],
            dtype='int64', length=2982),
 'HEAD_ANIMACY': Int64Index([-1, 0, 1, 2], dtype='int64'),
 'HEAD_CASE': Int64Index([-1, 0, 1, 2, 3, 4, 5, 6, 7], dtype='int64'),
 'HEAD_GENDER': Int64Index([-1, 0, 1, 2], dtype='int64'),
 'HEAD_POS': Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, -1, 14, 15, 16], dtype='int64'),
 'NUMBER': Int64Index([0, 1, -1, 2], dtype='int64'),
 'PERSON': Int64Index([0, -1, 1, 2], dtype=

In [57]:
df.loc[ (df['NER'] == 3.0 )]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
475,его,1,1,7.0,1.0,475.0,0,4,1,2,5,11,199,1,3,3,1,3,2.0,1,0.527685,0.353516,0.981266
566,он,1,1,9.0,3.0,566.0,0,1,1,2,5,12,231,0,0,1,0,3,536.0,1,0.511552,0.420731,0.981266
587,он,2,2,1.0,0.0,13.0,0,1,1,2,5,12,235,0,0,1,0,3,11.0,1,0.511552,0.538121,0.981266
594,он,2,2,1.0,1.0,20.0,0,1,1,2,5,12,242,0,0,0,0,3,11.0,1,0.511552,0.580780,0.981266
736,него,2,2,3.0,5.0,162.0,0,2,1,2,5,10,293,0,0,0,0,3,108.0,1,0.563433,0.594808,0.981266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14906,него,15,15,19.0,3.0,2314.0,0,2,1,2,5,21,2453,0,0,0,16,3,2295.0,1,0.563433,0.503185,0.940178
14967,он,16,16,1.0,4.0,33.0,0,1,1,2,5,12,2504,0,0,1,0,3,0.0,1,0.511552,0.615474,0.978046
16328,они,18,18,6.0,2.0,207.0,0,1,0,2,5,12,2507,0,0,0,9,3,0.0,0,0.468592,0.530037,0.986028
16391,его,18,18,6.0,5.0,270.0,0,4,1,2,5,1,1588,0,0,0,5,3,0.0,1,0.527685,0.525196,0.986028


In [58]:
df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 )]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
1,препарат,1,1,0.0,0.0,1.0,1,1,1,1,1,1,0,0,0,0,0,2,0.0,1,0.578114,0.544900,0.944452
2,рисперидон,1,1,0.0,0.0,2.0,2,1,1,1,1,2,1,1,1,1,1,1,2.0,1,0.741848,0.856890,0.944452
3,рисполепт,1,1,0.0,0.0,3.0,1,2,2,1,1,3,2,2,1,1,1,1,3.0,0,0.769299,0.561147,0.944452
4,сперидан,1,1,0.0,0.0,4.0,0,0,1,1,2,4,3,1,2,2,1,1,4.0,1,0.742337,0.620767,0.944452
21,лекарством,1,1,1.0,1.0,21.0,1,6,3,1,1,4,7,0,0,0,3,2,0.0,1,0.475286,0.595917,0.944452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17918,вел,21,21,4.0,2.0,137.0,0,0,1,1,0,0,2961,0,0,1,0,1,136.0,1,0.481298,0.397524,0.991343
17929,соф,21,21,4.0,3.0,148.0,1,2,2,1,1,21,2964,1,2,1,1,1,148.0,0,0.508576,0.504162,0.991343
17930,даке,21,21,4.0,3.0,149.0,1,3,2,1,1,21,2945,1,2,2,1,1,149.0,1,0.522228,0.502424,0.991343
17937,соф,21,21,4.0,3.0,156.0,1,2,2,1,1,12,2961,0,0,1,0,1,156.0,0,0.508576,0.397524,0.991343


Разделим датасет на тестовый и тренировочный. for_train, for_test 

In [126]:
# train, test = train_test_split(df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 ) | (df['NER'] == 3.0 )], test_size=0.2)
# train, test = train_test_split(df, test_size=0.2)
train, test = train_test_split(df.loc[(df['NER'] > 1 )], test_size=0.2)

In [127]:
train = train.reset_index()

In [128]:
test = test.reset_index()

In [129]:
test.head(5)

,index,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
0,4720,нормотимики,3,3,32.0,7.0,2852.0,2,1,1,1,1,22,1008,0,0,0,0,2,0.0,0,0.588630,0.540522,0.637320
1,454,препарат,1,1,7.0,0.0,454.0,1,1,1,1,1,12,189,0,0,0,0,2,2.0,1,0.578114,0.516539,0.981266
2,6216,его,5,5,1.0,0.0,18.0,0,4,1,2,5,1,73,0,0,0,0,3,14.0,1,0.527685,0.559254,0.929403
3,7082,препарат,5,5,28.0,10.0,884.0,1,1,1,1,1,1,1401,0,0,0,0,2,870.0,1,0.578114,0.519367,0.974590
4,21,лекарством,1,1,1.0,1.0,21.0,1,6,3,1,1,4,7,0,0,0,3,2,0.0,1,0.475286,0.595917,0.944452


Делаем наборы для обучения моделей.

In [130]:
X, y, X_test, y_test = train.drop(['HEAD_VECT', 'NUMBER', 'TOKEN',  'GENDER', 'POS', 'IS_ANSWER', 'PERSON', \
                                   'TOPIC_NUM', 'CASE',   'DEPENDENCY',   'HEAD_ANIMACY',	'HEAD_CASE',\
                                   	'HEAD_GENDER',  'ANIMACY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], \
train.COREFERENCE_CLUSTER, test.drop(['HEAD_VECT', 'NUMBER','TOKEN','GENDER', 'POS', 'IS_ANSWER', 'PERSON', 'TOPIC_NUM', \
                                      'CASE',  'DEPENDENCY',     'HEAD_ANIMACY',	'HEAD_CASE',\
                                      'HEAD_GENDER', 'ANIMACY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18],\
                                       test.COREFERENCE_CLUSTER

In [131]:
X.NER = [float(i) for i in X.NER]
X_test.NER = [float(i) for i in X_test.NER]
X.drop(['COREFERENCE_CLUSTER'], axis=1)

,index,POST_NUM,SENT_NUM,TOKEN_NUM,NER,TOKEN_VECT,SENT_VECT
0,2648,9.0,0.0,780.0,3.0,0.535635,0.940063
1,6273,4.0,0.0,75.0,3.0,0.511552,0.891926
2,429,6.0,0.0,429.0,2.0,0.578114,0.981266
3,956,7.0,2.0,382.0,2.0,0.578114,0.985002
4,9935,7.0,0.0,1396.0,2.0,0.548028,0.978340
...,...,...,...,...,...,...,...
285,8291,50.0,7.0,2093.0,3.0,0.535635,0.967372
286,5912,6.0,7.0,359.0,2.0,0.467804,0.953579
287,575,0.0,0.0,1.0,2.0,0.578114,0.981266
288,4578,31.0,1.0,2710.0,2.0,0.515678,0.853599


In [65]:
y

0       96.0
1      375.0
2      449.0
3      446.0
4      243.0
       ...  
285      0.0
286     27.0
287      0.0
288      0.0
289    446.0
Name: COREFERENCE_CLUSTER, Length: 290, dtype: float64

Обучаем модели.

In [66]:
clf = tree.DecisionTreeClassifier(max_depth=2000, )
clf.fit(X, y)
clf.score(X_test, y_test)

0.5753424657534246

In [67]:
clf_entropy = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf_entropy.fit(X, y)
clf_entropy.score(X_test, y_test)

0.5753424657534246

In [68]:
randf_clf = RandomForestClassifier(max_depth=3000, n_estimators=600, bootstrap=True, )
randf_clf.fit(X, y)
randf_clf.score(X_test, y_test)

0.6164383561643836

In [69]:

regr = linear_model.LinearRegression()
regr.fit(X, y)
regr.score(X_test, y_test)


1.0

In [70]:
svm_clf = svm.SVC(decision_function_shape='ovo', cache_size=500)
svm_clf.fit(X, y)
svm_clf.score(X_test, y_test)

0.3561643835616438

In [71]:

knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1).fit(X, y)
knn_clf.score(X_test, y_test)

0.5205479452054794

In [72]:
# from sklearn.naive_bayes import MultinomialNB
# naive_clf = MultinomialNB().fit(X, y)


# **ОЦЕНКА МОДЕЛЕЙ**

In [73]:
def my_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y_test[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0:
      if X_predicted[i] == y_test[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')



In [74]:
pred = clf.predict(X_test)
pred_randf = randf_clf.predict(X_test)
pred_entropy = clf_entropy.predict(X_test)
pred_regr = regr.predict(X_test)
pred_svm = svm_clf.predict(X_test)
pred_knn = knn_clf.predict(X_test)
# naive_pred = naive_clf.predict(X_test)

In [75]:
# my_eval(list(naive_pred), list(y_test))

In [76]:
my_eval(list(pred_knn), list(y_test))

17  true positive
31  false positive
21  true negative
4  false negative
---------
0.3541666666666667  presicion
0.8095238095238095  recall
0.4927536231884058  f-score


In [77]:
my_eval(list(pred_entropy), list(y_test))

16  true positive
30  false positive
26  true negative
1  false negative
---------
0.34782608695652173  presicion
0.9411764705882353  recall
0.5079365079365079  f-score


In [78]:
my_eval(list(pred), list(y_test))

16  true positive
30  false positive
26  true negative
1  false negative
---------
0.34782608695652173  presicion
0.9411764705882353  recall
0.5079365079365079  f-score


In [79]:
my_eval(list(pred_randf), list(y_test))

19  true positive
27  false positive
26  true negative
1  false negative
---------
0.41304347826086957  presicion
0.95  recall
0.5757575757575758  f-score


In [80]:
my_eval(list(pred_regr), list(y_test))

0  true positive
73  false positive
0  true negative
0  false negative
---------
0.0  presicion
0  recall
0  f-score


In [81]:
my_eval(list(pred_svm), list(y_test))

0  true positive
0  false positive
26  true negative
47  false negative
---------
0  presicion
0.0  recall
0  f-score


In [82]:
# df['TOPIC_NUM'] == 1
# df.loc[df['TOPIC_NUM'] == 1]

In [83]:
df.drop(['TOKEN','GENDER','NUMBER', 'POS', 'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', \
         'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',  'HEAD_ANIMACY',	'HEAD_CASE',	\
         'HEAD_GENDER', 'ANIMACY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18]

,TOKEN_NUM,NER,COREFERENCE_CLUSTER,TOKEN_VECT,SENT_VECT
0,0.0,0,0.0,0.507672,0.944452
1,1.0,2,0.0,0.578114,0.944452
2,2.0,1,2.0,0.741848,0.944452
3,3.0,1,3.0,0.769299,0.944452
4,4.0,1,4.0,0.742337,0.944452
...,...,...,...,...,...
17995,214.0,0,0.0,0.465665,0.989458
17996,215.0,0,0.0,0.594935,0.989458
17997,216.0,0,0.0,0.454485,0.989458
17998,217.0,0,0.0,0.525261,0.989458


In [84]:
len(df.loc[(df['POS'] == 5) & (df['NER'] > 1)].TOKEN)

120

In [85]:
df.loc[(df['COREFERENCE_CLUSTER'] != 0) & (df['NER'] > 1)]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
366,препарата,1,1,2.0,5.0,366.0,1,2,1,1,1,21,154,1,2,2,1,2,344.0,1,0.515678,0.417931,0.981266
393,препарата,1,1,3.0,1.0,393.0,1,2,1,1,1,21,164,1,4,3,1,2,382.0,1,0.515678,0.437745,0.981266
421,препарат,1,1,5.0,0.0,421.0,1,1,1,1,1,0,1,1,1,1,1,2,2.0,1,0.578114,0.856890,0.981266
429,препарат,1,1,6.0,0.0,429.0,1,1,1,1,1,12,179,0,0,0,0,2,2.0,1,0.578114,0.523989,0.981266
446,последнему,1,1,6.0,2.0,446.0,0,3,1,1,10,7,184,0,0,0,0,2,442.0,1,0.628062,0.530571,0.981266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14440,его,15,15,14.0,2.0,1848.0,0,4,1,2,5,1,1175,0,0,0,0,3,1827.0,1,0.527685,0.513876,0.972632
14478,него,15,15,14.0,6.0,1886.0,0,2,1,2,5,21,2453,0,0,0,16,3,1867.0,1,0.563433,0.503185,0.972632
14733,него,15,15,17.0,8.0,2141.0,0,2,1,2,5,10,2278,0,0,2,0,3,2138.0,1,0.563433,0.568975,0.973648
14906,него,15,15,19.0,3.0,2314.0,0,2,1,2,5,21,2453,0,0,0,16,3,2295.0,1,0.563433,0.503185,0.940178


In [175]:
int(df.loc[(df['TOPIC_NUM'] == 1) & (df['POS'] == 5)].values[0][4])
df.loc[(df['TOPIC_NUM'] == 1) & (df['POS'] == 5)].values[0][5]

9.0

# **ПОПАРНЫЙ МЕТОД**

In [86]:
def pair_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0:
      if X_predicted[i] == y[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')

In [230]:
df.loc[(df['TOPIC_NUM'] == 1) & \
              (df['NER'] == 1) & \
              (df['SENT_NUM'] <= 4.0) & \
              (df['POST_NUM'].isin([0.0, 2.0, 3.0]))]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,1,1,0.0,0.0,2.0,2,1,1,1,1,2,1,1,1,1,1,1,2.0,1,0.741848,0.856890,0.944452
3,рисполепт,1,1,0.0,0.0,3.0,1,2,2,1,1,3,2,2,1,1,1,1,3.0,0,0.769299,0.561147,0.944452
4,сперидан,1,1,0.0,0.0,4.0,0,0,1,1,2,4,3,1,2,2,1,1,4.0,1,0.742337,0.620767,0.944452
344,рисполепта,1,1,2.0,4.0,344.0,1,1,2,1,1,21,147,1,6,1,1,1,344.0,1,0.745356,0.502140,0.981266
382,рисперидон,1,1,3.0,0.0,382.0,2,1,1,1,1,1,67,0,0,0,0,1,382.0,1,0.741848,0.572582,0.981266
394,флюанксол,1,1,3.0,2.0,394.0,1,1,1,1,1,12,166,0,0,1,0,1,394.0,1,0.699340,0.467466,0.981266


In [288]:
topics = list(set(df.TOPIC_NUM.values))
topics


pair_anaphors = []
pair_antecedents = []
relation_list = []
counter = 0
ants_with_anaph = 0
len_anapfors = 0
for topic_num in topics:

  all_anaphors = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['NER'] > 1)]
  # all_anaphors = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['POS'] == 5)]

  all_antecedents = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['NER'] == 1)]
  
  len_anapfors += len(all_anaphors)
  for anaphor in all_anaphors.values:
    anapor_sent = anaphor[4]
    anapor_post = anaphor[3]
    local_possible_antecedents = df.loc[(df['TOPIC_NUM'] == topic_num) & \
                                        (df['NER'] == 1) & \
                                        (df['SENT_NUM'] <= anapor_sent) & \
                                        (df['POST_NUM'].isin([0.0, anapor_post]))]
    # чтобы примеры влезли в память, и модель не переобучилась на антипримерах,
    # возьмем один положительный и два отрицательных класса для каждого анафора
    counter = 0
    is_relation = 0
    for antecedent in local_possible_antecedents.values:
      if not antecedent.any():
        continue

      # print(anaphor[-5], antecedent[5])
      if anaphor[-5] == antecedent[5]:
        ants_with_anaph += 1
        is_relation = 1
        # такие срезы, чтобы убрать лишние данные, и в том числе разметку с ответами
        pair_anaphors.append(list(anaphor)[3:6]  + list(anaphor)[17:18]\
                              + list(anaphor)[19:22])
        
        pair_antecedents.append(list(antecedent)[3:6]  + list(antecedent)[17:18]\
                              + list(antecedent)[19:22])
        
        relation_list.append(is_relation)
      else:
        # if counter != 1:
        is_relation = 0

        pair_anaphors.append(list(anaphor)[3:6]  + list(anaphor)[17:18]\
                              + list(anaphor)[19:22])
        
        pair_antecedents.append(list(antecedent)[3:6]  + list(antecedent)[17:18]\
                              + list(antecedent)[19:22])
        
        relation_list.append(is_relation)
        counter = 1

  # counter += 1
print(len_anapfors)
print('---')
print(ants_with_anaph, 'ss')
print(pair_anaphors[0][0], pair_antecedents[0][0], relation_list[0])
len(pair_anaphors), len(pair_antecedents), len(relation_list)

363
---
188 ss
0.0 0.0 0


(1874, 1874, 1874)

In [289]:
# можно посмотреть, что получилось
relation_list[:20]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

In [290]:
pair_anaphors[0]

[0.0, 0.0, 1.0, 2, 1, 0.5781136751174927, 0.5449002981185913]

In [291]:
# проверим, третьим элементом должен быть номер токена
len([i for i in relation_list if i==1])

188

In [292]:
len([i for i in relation_list if i==0])

1686

In [293]:
# все сходится. мы взяли колонки с номерами поста, предложения и токена,\
# колонку NER, число, и вектора
df.loc[df.TOKEN_NUM == 21.0].iloc[[0]]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
21,лекарством,1,1,1.0,1.0,21.0,1,6,3,1,1,4,7,0,0,0,3,2,0.0,1,0.475286,0.595917,0.944452


In [294]:
cosine = [cosine_similarity([pair_anaphors[i]], [pair_antecedents[i]])[0][0] for i in range(len(pair_anaphors))]
# pair_df = pd.DataFrame(columns=['anaphs', 'ants', 'rel'])
pair_df = pd.DataFrame(columns=['cosine', 'rel'])

# pair_df['anaphs'] = pair_anaphors
# pair_df['ants'] = pair_antecedents
pair_df['cosine'] = cosine
pair_df['rel'] = relation_list

print(pair_df.loc[pair_df['cosine'] > 1])
print(pair_df.loc[pair_df['rel'] > 1])
pair_df 

Empty DataFrame
Columns: [cosine, rel]
Index: []
Empty DataFrame
Columns: [cosine, rel]
Index: []


,cosine,rel
0,0.848292,0
1,0.676185,0
2,0.693160,0
3,0.802308,0
4,0.939287,0
...,...,...
1869,0.999987,0
1870,0.999987,0
1871,0.999996,0
1872,0.999996,0


In [295]:
pair_df.loc[pair_df.rel != 0]

,cosine,rel
16,0.999994,1
20,0.999993,1
21,0.744614,1
24,0.744280,1
31,0.999998,1
...,...,...
1777,0.999998,1
1810,0.999998,1
1825,0.999999,1
1827,0.999999,1


In [296]:
# train, test = train_test_split(med_df, test_size=0.3, shuffle=True)
# train = train.reset_index()
# test = test.reset_index()

train, test = train_test_split(pair_df, test_size=0.3, shuffle=True)
train = train.reset_index()
test = test.reset_index()


In [297]:
train

,index,cosine,rel
0,571,0.803847,0
1,174,0.932367,0
2,403,0.999998,0
3,677,0.782786,0
4,1548,0.868781,0
...,...,...,...
1306,1286,0.999992,1
1307,810,0.999933,0
1308,1785,1.000000,0
1309,311,0.804074,0


In [298]:
train.iloc[:1, 1:2]

,cosine
0,0.803847


In [299]:
test.iloc[:1, 1:2]

,cosine
0,0.998968


In [300]:
X, y, X_test, y_test = train.iloc[:, 1:2], \
                    train.rel, test.iloc[:, 1:2], test.rel

In [301]:
def pair_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0:
      if X_predicted[i] == y[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')

In [302]:
regr = linear_model.LinearRegression()
regr.fit(X, y)


print('-----')
pred = regr.predict(X_test)
pair_eval(pred, y_test)

-----
0  true positive
563  false positive
0  true negative
0  false negative
---------
0.0  presicion
0  recall
0  f-score


In [303]:
knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1)
knn_clf.fit(X, y)


print('-----')
pred = knn_clf.predict(X_test)
pair_eval(pred, y_test)

-----
8  true positive
23  false positive
483  true negative
49  false negative
---------
0.25806451612903225  presicion
0.14035087719298245  recall
0.1818181818181818  f-score


In [304]:
dtclf = tree.DecisionTreeClassifier(max_depth=2000, )
dtclf.fit(X, y)


print('-----')
pred = dtclf.predict(X_test)
pair_eval(pred, y_test)


-----
6  true positive
13  false positive
493  true negative
51  false negative
---------
0.3157894736842105  presicion
0.10526315789473684  recall
0.15789473684210525  f-score


In [305]:
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf.fit(X, y)


print('-----')
pred = clf.predict(X_test)
pair_eval(pred, y_test)

-----
6  true positive
13  false positive
493  true negative
51  false negative
---------
0.3157894736842105  presicion
0.10526315789473684  recall
0.15789473684210525  f-score


In [306]:
clf = RandomForestClassifier(max_depth=3000, n_estimators=600, bootstrap=True, )
clf.fit(X, y)


print('-----')
pred = clf.predict(X_test)
rule_eval(pred, y_test)

-----
6  true positive
14  false positive
492  true negative
51  false negative
---------
0.3  presicion
0.10526315789473684  recall
0.1558441558441558  f-score


In [307]:
# knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1).fit(X, y)
# knn_clf.score(X_test, y_test)

In [106]:
# clf = tree.DecisionTreeClassifier(max_depth=2000, )
# clf.fit(X, y)
# clf.score(X_test, y_test)

# **Извлечение номинативных анафоров**

In [107]:
df.loc[(df['POS'] == 1) &  (df['NER'] == 0)].iloc[:209, :]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
16,проблем,1,1,1.0,1.0,16.0,1,2,2,1,1,12,7,0,0,0,3,0,0.0,0,0.539869,0.595917,0.944452
19,частности,1,1,1.0,1.0,19.0,1,5,2,1,1,15,10,0,0,0,4,0,0.0,1,0.550824,0.420717,0.944452
24,мере,1,1,1.0,2.0,24.0,1,3,2,1,1,15,12,0,0,0,4,0,0.0,1,0.540037,0.548893,0.944452
25,врач,1,1,1.0,2.0,25.0,2,1,1,1,1,12,11,0,0,0,0,0,0.0,1,0.614582,0.505191,0.944452
35,детства,1,1,1.0,3.0,35.0,1,2,3,1,1,10,15,0,0,0,0,0,0.0,1,0.504010,0.334011,0.944452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
849,месяц,2,2,5.0,1.0,275.0,1,4,1,1,1,10,324,0,0,0,5,0,0.0,1,0.595416,0.500339,0.985002
853,манию,2,2,5.0,1.0,279.0,1,4,2,1,1,10,326,0,0,2,0,0,0.0,1,0.532399,0.547506,0.985002
856,мг,2,2,5.0,1.0,282.0,1,1,1,1,1,10,326,0,0,2,0,0,0.0,1,0.538409,0.547506,0.985002
859,началом,2,2,5.0,2.0,285.0,1,6,3,1,1,10,328,0,0,2,0,0,0.0,1,0.527950,0.443062,0.985002


In [108]:
df.loc[(df['POS'] == 1) & (df['NER'] > 1)] 

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
1,препарат,1,1,0.0,0.0,1.0,1,1,1,1,1,1,0,0,0,0,0,2,0.0,1,0.578114,0.544900,0.944452
21,лекарством,1,1,1.0,1.0,21.0,1,6,3,1,1,4,7,0,0,0,3,2,0.0,1,0.475286,0.595917,0.944452
217,таблетки,1,1,1.0,21.0,217.0,1,4,2,1,1,10,101,0,0,2,0,2,0.0,0,0.516602,0.702134,0.981266
295,таблетки,1,1,2.0,0.0,295.0,1,4,2,1,1,10,101,0,0,2,0,2,0.0,0,0.516602,0.702134,0.981266
366,препарата,1,1,2.0,5.0,366.0,1,2,1,1,1,21,154,1,2,2,1,2,344.0,1,0.515678,0.417931,0.981266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15651,вакцина,16,16,16.0,0.0,717.0,1,1,2,1,1,21,2642,1,1,3,1,2,0.0,1,0.573031,0.478954,0.944977
15692,химиотерапии,16,16,17.0,3.0,758.0,1,2,2,1,1,21,142,2,1,1,1,2,0.0,1,0.582935,0.592626,0.940661
15751,химиотерапий,16,16,18.0,1.0,817.0,1,2,2,1,1,21,2609,1,2,2,1,2,0.0,0,0.635564,0.658113,0.932694
15873,вакцина,16,16,19.0,0.0,939.0,1,1,2,1,1,21,2642,1,1,3,1,2,0.0,1,0.573031,0.478954,0.743826


In [109]:
df.loc[(df['POS'] == 1) & (df['NER'] > 1)].TOKEN.values

array(['препарат', 'лекарством', 'таблетки', 'таблетки', 'препарата',
       'препарата', 'препарат', 'препарат', 'препарат', 'препарат',
       'препарат', 'препарата', 'препарат', 'препарат', 'препарата',
       'препарат', 'препарат', 'препарата', 'препарата', 'препарат',
       'препарат', 'препарата', 'сибазона', 'препарат', 'препарат',
       'препарата', 'лекарства', 'препаратов', 'правозащитники',
       'препарата', 'препарата', 'препарат', 'препарат', 'препарата',
       'лекарства', 'препаратов', 'препарата', 'препарата', 'таблетки',
       'препарата', 'препарат', 'препараты', 'препарата', 'препараты',
       'антипсихотиком', 'препаратов', 'препараты', 'антидепресантом',
       'препаратов', 'препарат', 'препараты', 'лекарств', 'препарата',
       'препарат', 'препарата', 'препарат', 'нейролептика', 'нормотимики',
       'препараты', 'дженериков', 'препарата', 'препарат', 'препарат',
       'средство', 'нейролептиков', 'препарату', 'препарату',
       'препаратов', 'препар

In [110]:
# df.loc[(df['POS'] == 1) &  (df['NER'] == 0)].iloc[:209, :].append(df.loc[(df['POS'] == 1) & (df['NER'] > 1)])
df.loc[(df['POS'] == 1) & (df['NER'] > 1)]
np.mean(df.loc[(df['POS'] == 1) & (df['NER'] > 1)].CASE)

2.0669642857142856

In [111]:
print(len(df.loc[(df['POS'] != 1) & (df['POS'] != 5) & (df['NER'] > 1) ]))
df.loc[(df['POS'] != 1) & (df['POS'] != 5) & (df['NER'] > 1) ]

19


,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
446,последнему,1,1,6.0,2.0,446.0,0,3,1,1,10,7,184,0,0,0,0,2,442.0,1,0.628062,0.530571,0.981266
3539,препарат.точнее,3,3,21.0,0.0,1671.0,0,0,0,1,13,0,691,0,0,0,15,2,3.0,2,0.633332,0.866844,0.955688
5222,новейший,3,3,38.0,5.0,3354.0,0,1,1,1,10,0,1085,0,1,1,8,2,3350.0,1,0.612320,0.589786,0.945780
6857,тиапридал,5,5,23.0,6.0,659.0,0,0,1,1,0,12,958,0,0,0,0,2,659.0,1,0.702556,0.520091,0.922783
8834,это,7,7,0.0,25.0,295.0,0,0,0,1,3,12,941,0,0,0,9,3,293.0,2,0.475817,0.455471,0.867473
8928,хт,7,7,1.0,2.0,389.0,0,0,0,1,16,21,1634,1,1,1,1,2,0.0,2,0.548028,0.501892,0.907328
8987,хт,7,7,1.0,8.0,448.0,0,0,0,1,16,21,1651,1,2,3,1,2,0.0,2,0.548028,0.545719,0.907328
9086,хт,7,7,1.0,13.0,547.0,0,0,0,1,16,10,229,0,0,0,5,2,0.0,2,0.548028,0.573037,0.939133
9497,ксф,7,7,1.0,57.0,958.0,0,0,0,1,16,12,1791,0,0,0,0,2,0.0,2,0.517060,0.649600,0.978340
9935,хт,7,7,7.0,0.0,1396.0,0,0,0,1,16,21,1634,1,1,1,1,2,0.0,2,0.548028,0.501892,0.978340


In [112]:
med_df = df.loc[(df['POS'] == 1) &  (df['NER'] == 0)].iloc[:2409, :]\
            .append(df.loc[(df['POS'] == 1) & (df['NER'] > 1)]).\
            loc[:,['TOKEN_VECT', 'CASE', 'HEAD_VECT', 'NER']]


# med_df = df.loc[(df['POS'] == 1) &  (df['NER'] == 0)].iloc[:1409, :]\
#             .append(df.loc[(df['POS'] == 1) & (df['NER'] > 1)]).\
#             loc[:,['TOKEN_VECT', 'HEAD_VECT', 'SENT_VECT', 'NER']]

In [113]:
med_df

,TOKEN_VECT,CASE,HEAD_VECT,NER
16,0.539869,2,0.595917,0
19,0.550824,5,0.420717,0
24,0.540037,3,0.548893,0
25,0.614582,1,0.505191,0
35,0.504010,2,0.334011,0
...,...,...,...,...
15651,0.573031,1,0.478954,2
15692,0.582935,2,0.592626,2
15751,0.635564,2,0.658113,2
15873,0.573031,1,0.478954,2


In [114]:
train, test = train_test_split(med_df, test_size=0.3, shuffle=True)
train = train.reset_index().drop(['index'], axis=1)
test = test.reset_index().drop(['index'], axis=1)
X, y, X_test, y_test = train.drop(['NER'], axis=1), \
                    train.NER, test.drop(['NER'], axis=1), test.NER

In [115]:
# pred = knn_clf.predict(X_test)
# pair_eval(pred, y_test)

In [116]:
X

,TOKEN_VECT,CASE,HEAD_VECT
0,0.643746,4,0.464462
1,0.435717,5,0.489349
2,0.627453,5,0.655785
3,0.471773,1,0.528794
4,0.465665,2,0.541332
...,...,...,...
1838,0.508487,1,0.581418
1839,0.570073,2,0.512839
1840,0.516946,2,0.506819
1841,0.480806,7,0.471547


In [117]:
y

0       0
1       0
2       0
3       0
4       0
       ..
1838    2
1839    0
1840    0
1841    0
1842    0
Name: NER, Length: 1843, dtype: int64

In [118]:
len([i for i in y if i == 2])

156

In [119]:
y_test

0      0
1      0
2      0
3      0
4      0
      ..
785    0
786    0
787    2
788    0
789    0
Name: NER, Length: 790, dtype: int64

In [120]:
len([i for i in y_test if i == 2])

66

In [121]:
regr = linear_model.LinearRegression()
regr.fit(X, y)


print('-----')
pred = regr.predict(X_test)
pair_eval(pred, y_test)

-----
0  true positive
790  false positive
0  true negative
0  false negative
---------
0.0  presicion
0  recall
0  f-score


In [122]:
knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1)
knn_clf.fit(X, y)


print('-----')
pred = knn_clf.predict(X_test)
pair_eval(pred, y_test)

-----
41  true positive
18  false positive
706  true negative
25  false negative
---------
0.6949152542372882  presicion
0.6212121212121212  recall
0.656  f-score


In [123]:
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf.fit(X, y)


print('-----')
pred = clf.predict(X_test)
pair_eval(pred, y_test)

-----
50  true positive
23  false positive
700  true negative
17  false negative
---------
0.684931506849315  presicion
0.746268656716418  recall
0.7142857142857143  f-score
